This code defines a ResNet50 architecture using TensorFlow's Keras API. ResNet50 is a convolutional neural network model that consists of 50 layers, known for its effectiveness in image classification tasks.

Here's a breakdown of the code:

1. **Imports**: The code imports necessary modules from TensorFlow, including `tf` for TensorFlow itself and specific modules like `Model` and various layers from `tensorflow.keras.models`.

2. **Identity Block Function**: `identity_block` function defines a block in the ResNet architecture where the input tensor is passed through a series of convolutional layers, with the addition of the original input tensor to the final output. This function is used for residual connections within the blocks.

3. **Convolutional Block Function**: `conv_block` function defines a block similar to `identity_block`, but with a convolutional layer with stride 2 at the beginning. This function is used when the input and output dimensions of the block are different.

4. **ResNet50 Function**: `resnet50` function constructs the entire ResNet50 architecture. It starts with an input layer followed by a convolutional layer and max-pooling layer. Then, it builds several convolutional blocks and identity blocks, forming the main body of the network. Finally, it adds a global average pooling layer and a fully connected layer with softmax activation for classification.

5. **Model Creation**: The ResNet50 model is created by calling the `resnet50` function.

6. **Model Summary**: The summary of the model, showing the layers, output shapes, and number of parameters, is printed out.

In [37]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Conv2D, UpSampling2D, Concatenate, Activation
from tensorflow.keras.models import Model

In [38]:
def resnet(input_shape, num_classes):
    base_model = ResNet50(input_shape=input_shape, include_top=False)

    # Encoder
    encoder = tf.keras.Model(inputs=base_model.input, outputs=base_model.get_layer('conv4_block6_out').output)

    # Decoder
    skip_connections = ["conv3_block4_out", "conv2_block3_out", "conv1_relu"]
    encoder_output = encoder.output

    upconv1 = UpSampling2D()(encoder_output)
    skip_connection1 = base_model.get_layer(skip_connections[0]).output
    merge1 = Concatenate()([upconv1, skip_connection1])
    conv1 = Conv2D(512, (3, 3), padding='same')(merge1)
    conv1 = Activation('relu')(conv1)
    conv1 = Conv2D(512, (3, 3), padding='same')(conv1)
    conv1 = Activation('relu')(conv1)

    upconv2 = UpSampling2D()(conv1)
    skip_connection2 = base_model.get_layer(skip_connections[1]).output
    merge2 = Concatenate()([upconv2, skip_connection2])
    conv2 = Conv2D(256, (3, 3), padding='same')(merge2)
    conv2 = Activation('relu')(conv2)
    conv2 = Conv2D(256, (3, 3), padding='same')(conv2)
    conv2 = Activation('relu')(conv2)

    upconv3 = UpSampling2D()(conv2)
    skip_connection3 = base_model.get_layer(skip_connections[2]).output
    merge3 = Concatenate()([upconv3, skip_connection3])
    conv3 = Conv2D(128, (3, 3), padding='same')(merge3)
    conv3 = Activation('relu')(conv3)
    conv3 = Conv2D(128, (3, 3), padding='same')(conv3)
    conv3 = Activation('relu')(conv3)

    upconv4 = UpSampling2D()(conv3)
    conv4 = Conv2D(64, (3, 3), padding='same')(upconv4)
    conv4 = Activation('relu')(conv4)
    conv4 = Conv2D(64, (3, 3), padding='same')(conv4)
    conv4 = Activation('relu')(conv4)

    # Output layer
    output = Conv2D(num_classes, (1, 1), activation='softmax')(conv4)

    model = Model(inputs=base_model.input, outputs=output)
    return model

In [39]:
if __name__ == "__main__":
    input_shape = (256, 256, 3)
    num_classes = 2
    model = resnet(input_shape, num_classes)
    model.save('resnet.keras')